In [1]:
import os
from utils import *
from conv_util import *
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
repeat_size = 81

In [85]:
# Misc
LOG_DIR = './log/'

# Training Parameters
learning_rate = 0.001
num_epochs = 100
num_steps = 1000
batch_size = 256
display_step = 50

# Network Parameters
num_classes = 6
keep_rate = 1

In [86]:
raw = pd.read_csv('./vultures.csv', header=None)

In [92]:
mean = np.array([0. for i in  range(3)])
X = []
Y = []

for i in range(len(raw)):
    
    row = np.array(raw.loc[i][~raw.loc[i].isnull()])
    data = row[0:-1].astype(float)
    label = row[-1]
    
    x_axis = data[0::3]
    y_axis = data[1::3]
    z_axis = data[2::3]
    
    x_axis = repeat_crop_data (x_axis, repeat_size)
    y_axis = repeat_crop_data (y_axis, repeat_size)
    z_axis = repeat_crop_data (z_axis, repeat_size)
    
    mean[0] += x_axis.mean()
    mean[1] += y_axis.mean()
    mean[2] += z_axis.mean()
    
    X.append(np.stack((x_axis, y_axis, z_axis)))#.reshape((9, 9, 3)))
    Y.append(label)

mean = mean/len(raw)
    
X = np.array(X).reshape((-1, 9, 9, 3))
Y = np.array(Y)
label_names = np.unique(Y)


le = preprocessing.LabelEncoder()
le.fit(np.unique(Y))
Y = le.transform(Y)

In [93]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [94]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    keep_prob = tf.placeholder(tf.float32)
    
    # Dataset Preparation
    trn_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(128).repeat().batch(batch_size)
    test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).shuffle(128).repeat().batch(batch_size)
    
    iterator = tf.data.Iterator.from_structure(trn_ds.output_types, trn_ds.output_shapes)
    x, y = iterator.get_next()
    
    training_init = iterator.make_initializer(trn_ds)
    testing_init = iterator.make_initializer(test_ds)
    
    # Logits and Predictions
    logits = build_model(x, keep_prob, input_dim=[9, 9], mean=mean) 
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y))
        tf.summary.scalar('cross_entropy_loss', loss_op)
    
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], y)
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Global Initializer
    initializer = tf.global_variables_initializer()
    
    # Summary
    summary = tf.summary.merge_all()
    
    # Global saver
    saver = tf.train.Saver()

In [95]:
MODEL_ID = 'vulture_plain'

In [96]:
with tf.Session(graph=model_graph) as sess:
    
    # Run Global Initializer
    sess.run(initializer)
    
    sess.run(training_init)
    for step in range(1, num_steps+1):
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={keep_prob: keep_rate})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    sess.run(testing_init)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={keep_prob: 1.0}))
    
    # Save model
    saver.save(sess, os.path.join(LOG_DIR, 'model_%s'%MODEL_ID, "final.model.ckpt"), step)

Step 1, Minibatch Loss= 17.0872, Training Accuracy= 0.418
Step 50, Minibatch Loss= 0.3319, Training Accuracy= 0.906
Step 100, Minibatch Loss= 0.0463, Training Accuracy= 0.996
Step 150, Minibatch Loss= 0.0175, Training Accuracy= 1.000
Step 200, Minibatch Loss= 0.0149, Training Accuracy= 0.996
Step 250, Minibatch Loss= 0.0087, Training Accuracy= 1.000
Step 300, Minibatch Loss= 0.0043, Training Accuracy= 1.000
Step 350, Minibatch Loss= 0.0032, Training Accuracy= 1.000
Step 400, Minibatch Loss= 0.0017, Training Accuracy= 1.000
Step 450, Minibatch Loss= 0.0006, Training Accuracy= 1.000
Step 500, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Step 550, Minibatch Loss= 0.0007, Training Accuracy= 1.000
Step 600, Minibatch Loss= 0.0005, Training Accuracy= 1.000
Step 650, Minibatch Loss= 0.0004, Training Accuracy= 1.000
Step 700, Minibatch Loss= 0.0003, Training Accuracy= 1.000
Step 750, Minibatch Loss= 0.0003, Training Accuracy= 1.000
Step 800, Minibatch Loss= 0.0002, Training Accuracy= 1.000

In [97]:
with tf.Session(graph=model_graph) as sess:
    
    # Restore pretrained model
    saver.restore(sess, './log/model_vulture_plain/final.model.ckpt-1000')
    print('Model restored.')
    
    sess.run(testing_init)
    print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={keep_prob: 1.0}))
    

INFO:tensorflow:Restoring parameters from ./log/model_vulture_plain/final.model.ckpt-1000
Model restored.
Testing Accuracy: 0.734375


In [46]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.1)

train_x = train_x.astype(np.float32)
test_x = test_x.astype(np.float32)

In [47]:
MODEL_ID = 'vulture_fine_tune'
num_steps = 500
learning_rate = 0.0001

In [48]:
with tf.Session(graph=model_graph) as sess:
    
    # Restore pretrained model
    saver.restore(sess, './log/model_storkes_plain/final.model.ckpt-1000')
    print('Model restored.')
    
    sess.run(training_init)
    for step in range(1, num_steps+1):
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={keep_prob: keep_rate})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    sess.run(testing_init)
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={keep_prob: 1.0}))
    
    # Save model
    saver.save(sess, os.path.join(LOG_DIR, 'model_%s'%MODEL_ID, "final.model.ckpt"), step)

INFO:tensorflow:Restoring parameters from ./log/model_storkes_plain/final.model.ckpt-1000
Model restored.
Step 1, Minibatch Loss= 42.1347, Training Accuracy= 0.082
Step 50, Minibatch Loss= 0.6939, Training Accuracy= 0.793
Step 100, Minibatch Loss= 0.3468, Training Accuracy= 0.918
Step 150, Minibatch Loss= 0.2156, Training Accuracy= 0.957
Step 200, Minibatch Loss= 0.1541, Training Accuracy= 0.953
Step 250, Minibatch Loss= 0.1459, Training Accuracy= 0.965
Step 300, Minibatch Loss= 0.0826, Training Accuracy= 0.977
Step 350, Minibatch Loss= 0.0852, Training Accuracy= 0.980
Step 400, Minibatch Loss= 0.0675, Training Accuracy= 0.984
Step 450, Minibatch Loss= 0.0447, Training Accuracy= 0.996
Step 500, Minibatch Loss= 0.0319, Training Accuracy= 0.992
Optimization Finished!
Testing Accuracy: 0.71875


In [94]:
with tf.Session(graph=model_graph) as sess:
    
    # Restore pretrained model
    saver.restore(sess, './log/model_vulture_fine_tune/final.model.ckpt-1000')
    print('Model restored.')
    
    sess.run(testing_init)
    print("Testing Accuracy:", \
    sess.run(accuracy, feed_dict={keep_prob: 1.0}))

INFO:tensorflow:Restoring parameters from ./log/model_vulture_fine_tune/final.model.ckpt-1000
Model restored.
Testing Accuracy: 0.7578125
